In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
import matrix_dev

In [8]:
raw_docs = matrix_dev.matrix_dev('basic_train').raw_docs()
dev = matrix_dev.matrix_dev('basic_dev').raw_docs()

In [3]:
vocab = set()
for doc in raw_docs:
    for word in list(doc):
        vocab.add(word)

In [4]:
no_features = 1000
tf_vectorizer = TfidfVectorizer(max_features=no_features) #stop_words='english')
tf = tf_vectorizer.fit_transform(raw_docs)
tf_feature_names = tf_vectorizer.get_feature_names()

In [5]:
lda = LatentDirichletAllocation(n_topics=5, max_iter=3)
lda.fit(tf)

/Users/nantanick/anaconda/envs/nlu/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/Users/nantanick/anaconda/envs/nlu/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=3,
             mean_change_tol=0.001, n_components=10, n_jobs=1, n_topics=5,
             perp_tol=0.1, random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [11]:
X_test = tf_vectorizer.fit_transform(dev)
doc_topic_dist_unnorm = np.matrix(lda.transform(X_test))

In [12]:
doc_topic_dist = doc_topic_dist_unnorm/doc_topic_dist_unnorm.sum(axis=1)